# Simple GenAI App Using Langchain 

### load data --> Docs ---> divided our text into chaunks---> vecotre-->vectore Embedding----> vectore store in DB

### General Rule to Identify Module

- 1. **langchain_core (CORE LAYER)
Iska kaam: Sirf basic building blocks provide karna, jo LangChain ke andar hi kaam karte hain.**

- 2. **langchain_community (INTEGRATIONS LAYER)
Iska kaam: Saare external integrations handle karna.**

- 3. **langchain (HIGH-LEVEL LAYER)
Iska kaam: Sirf user-friendly wrappers provide karna jo core + community ko combine karte hain.**
    - **Agar integration hai (jaise API, database, file format) → langchain_community**
    - **Agar prompt, chain, schema, message → langchain_core**
    - **Agar high-level chains ya wrappers → langchain**



In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
### Data Ingestion = from the website scrape tha data
from langchain_community.document_loaders import WebBaseLoader

In [21]:
loader = WebBaseLoader("https://www.britannica.com/biography/Mahatma-Gandhi")
loader

In [22]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.britannica.com/biography/Mahatma-Gandhi', 'title': 'Mahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, Gandhi Jayanti, & Facts | Britannica', 'description': 'Mahatma Gandhi (1869–1948) was an Indian lawyer, politician, social activist, and writer who became the leader of the Indian Independence Movement against British rule. He is internationally esteemed for his doctrine of nonviolent protest (satyagraha) to achieve political and social progress.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\nMahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, Gandhi Jayanti, & Facts | Britannica\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch Britannica\n\n\n\n\n\nClick here to search\n\n\n\n\n\n\n\n\n\n\n\nSearch Britannica\n\n\n\n\n\nClick here to search\n\n\n\n\n\n\n\n   SUBSCRIBE\n\n\n\n   SUBSCRIBE\n\n\nLogin\n\nhttps://premium.br

In [23]:
## load data ---> Docs---> divide document into chaunk document---> vecotr-->vectore Embedding----> vectorestore in DB

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document  = text_splitter.split_documents(docs)
document

[Document(metadata={'source': 'https://www.britannica.com/biography/Mahatma-Gandhi', 'title': 'Mahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, Gandhi Jayanti, & Facts | Britannica', 'description': 'Mahatma Gandhi (1869–1948) was an Indian lawyer, politician, social activist, and writer who became the leader of the Indian Independence Movement against British rule. He is internationally esteemed for his doctrine of nonviolent protest (satyagraha) to achieve political and social progress.', 'language': 'en'}, page_content='Mahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, Gandhi Jayanti, & Facts | Britannica\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch Britannica\n\n\n\n\n\nClick here to search\n\n\n\n\n\n\n\n\n\n\n\nSearch Britannica\n\n\n\n\n\nClick here to search\n\n\n\n\n\n\n\n   SUBSCRIBE\n\n\n\n   SUBSCRIBE\n\n\nLogin\n\nhttps://premium.britannica.com/premium-members

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

In [25]:
from langchain_community.vectorstores import FAISS
vecotrStoredb = FAISS.from_documents(document,embedding)

In [ ]:
text = "what is all You "

In [26]:
# input ----> Retriever ----> vectorestoredb
vecotrStoredb

In [27]:
## Query from vectorstore db
query = "Sojourn in England and return to India of Mahatma Gandhi"
result = vecotrStoredb.similarity_search(query)
result[0].page_content

'Meanwhile, his family was debating his future. Left to himself, he would have liked to have been a doctor. But, besides the Vaishnava prejudice against vivisection, it was clear that, if he was to keep up the family tradition of holding high office in one of the states in Gujarat, he would have to qualify as a barrister. That meant a visit to England, and Mohandas, who was not too happy at Samaldas College, jumped at the proposal. His youthful imagination conceived England as “a land of philosophers and poets, the very centre of civilization.” But there were several hurdles to be crossed before the visit to England could be realized. His father had left the family little property; moreover, his mother was reluctant to expose her youngest child to unknown temptations and dangers in a distant land. But Mohandas was determined to visit England. One of his brothers raised the necessary money, and his mother’s doubts were allayed when he took a vow that, while away from home, he would not'

In [28]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.1-8b-instant"
)

## Retriver Chain , Document Chain 

- The create_stuff_documents_chain function in LangChain is used to create a chain that passes a list of documents to a language model (LLM) by combining them into a single context string.

- 1) Documents ka text uthata hai
- 2) Sab documents ko prompt ke andar stuff (combine) karta hai
- 3) Combined prompt → LLM ko deta hai

In [29]:
## Retriver Chain , Document Chain 
# "create_stuff_documents_chain":=is context information given of any document 

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following question based only on the provided context."),
                ("user", "<context>{context}</context>\n\nQuestion: {input}")
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)  # prompt to connect with llm "they are combine form"
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='<context>{context}</context>\n\nQuestion: {input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B0EC835000>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B0EC835300>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputP

In [30]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Sojourn in England and return to India of Mahatma Gandhi",
    "context":[Document(page_content="Sojourn in England and return to India of Mahatma Gandhi")]
})

"I can tell you more about the sojourn in England and return to India of Mahatma Gandhi, considering that this is a specific context.\n\nMahatma Gandhi, also known as Mohandas Karamchand Gandhi, was a key figure in India's struggle for independence from British rule. He studied law in England from 1888 to 1891, which was his sojourn in England.\n\nDuring his time in England, Gandhi was exposed to various social and political movements, including the ideas of Henry David Thoreau and Leo Tolstoy, which significantly influenced his thoughts on non-violent resistance and civil disobedience.\n\nIn 1891, Gandhi returned to India and began his practice as a lawyer in Bombay. However, he soon became involved in the Indian independence movement and was imprisoned several times for his activism.\n\nGandhi's return to India marked the beginning of his long and tumultuous journey towards leading India to independence, which was finally achieved in 1947."

#### Ye code ek document ko chain me pass karta hai. Chain document ke text ko context ke roop me LLM ko bhejta hai, aur phir LLM us context ke base par answer generate karke return karta hai.

In [31]:
### input ----> Retriever ----> vectorsotredb
vecotrStoredb

#### create_retrieval_chain
- 1️⃣ User question aata hai
- 2️⃣ retriever → relevant documents nikalta hai
- 3️⃣ Documents → document_chain ko jate hain
- 4️⃣ document_chain → LLM se answer banwata hai

In [32]:
retriever =vecotrStoredb.as_retriever()

from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, document_chain)

In [33]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B0EC50D1E0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(in

In [34]:
### GEt the responce from the LLM
response = retrieval_chain.invoke({
    "input":"Sojourn in England and return to India of Mahatma Gandhi"
})
print(response["answer"])

According to the provided context, Mahatma Gandhi's sojourn in England and his return to India can be described as follows:

Initially, Gandhi's family was debating his future, and it was decided that he would visit England to qualify as a barrister. This was necessary to keep up the family tradition of holding high office in one of the states in Gujarat. Gandhi's youthful imagination had a romanticized view of England as a "land of philosophers and poets, the very centre of civilization."

However, there were several hurdles to be crossed before Gandhi could visit England. His father had left the family little property, and his mother was reluctant to expose her youngest child to unknown temptations and dangers in a distant land. But Gandhi was determined to visit England and eventually, with the help of one of his brothers, he managed to raise the necessary money to fund his trip.

Gandhi's mother's doubts were allayed when he took a vow to remain chaste and not get involved in any r

In [35]:
response

{'input': 'Sojourn in England and return to India of Mahatma Gandhi',
 'context': [Document(id='7c13869a-231d-4bb4-ae92-24836db10a0b', metadata={'source': 'https://www.britannica.com/biography/Mahatma-Gandhi', 'title': 'Mahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, Gandhi Jayanti, & Facts | Britannica', 'description': 'Mahatma Gandhi (1869–1948) was an Indian lawyer, politician, social activist, and writer who became the leader of the Indian Independence Movement against British rule. He is internationally esteemed for his doctrine of nonviolent protest (satyagraha) to achieve political and social progress.', 'language': 'en'}, page_content='Meanwhile, his family was debating his future. Left to himself, he would have liked to have been a doctor. But, besides the Vaishnava prejudice against vivisection, it was clear that, if he was to keep up the family tradition of holding high office in one of the states in Gujarat, he would have to qualify as a barrister. 

In [36]:
response['context']

[Document(id='7c13869a-231d-4bb4-ae92-24836db10a0b', metadata={'source': 'https://www.britannica.com/biography/Mahatma-Gandhi', 'title': 'Mahatma Gandhi | Biography, Education, Religion, Accomplishments, Death, Gandhi Jayanti, & Facts | Britannica', 'description': 'Mahatma Gandhi (1869–1948) was an Indian lawyer, politician, social activist, and writer who became the leader of the Indian Independence Movement against British rule. He is internationally esteemed for his doctrine of nonviolent protest (satyagraha) to achieve political and social progress.', 'language': 'en'}, page_content='Meanwhile, his family was debating his future. Left to himself, he would have liked to have been a doctor. But, besides the Vaishnava prejudice against vivisection, it was clear that, if he was to keep up the family tradition of holding high office in one of the states in Gujarat, he would have to qualify as a barrister. That meant a visit to England, and Mohandas, who was not too happy at Samaldas Col